In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import shuffle

os.chdir(r"F:\\KARTIK\\2021\\Freezing of Gait")

In [2]:
data_path = os.path.join(os.getcwd(), 'dataset_fog_release', 'dataset', 'FINAL FEATURES', 'final_ftrs_size_1.5_3.csv')
if os.path.exists(data_path):
    print(True)

True


In [3]:
df = pd.read_csv(data_path)
df.head()

,Unnamed: 0,0,Minimum_A_F,Maximum_A_F,RMS_A_F,Variance_A_F,STD_A_F,RangeA_F,Minimum_A_V,Maximum_A_V,RMS_A_V,Variance_A_V,RangeA_V,Minimum_A_L,RMS_A_L,Variance_A_L,RangeA_L,EN_A_F,EN_A_V,EN_A_L
0,8203,0,-606,1000,379.140578,94373.349370,307.202457,1606,431,1529,1105.287236,74150.264650,1098,-465,257.441893,25377.585690,999,9199845,78186232,4241685
1,6414,0,131,171,148.916084,77.319336,8.793141,40,960,1000,981.148084,106.616211,40,306,324.583002,94.433594,50,1419264,61609700,6742664
2,7376,0,-888,858,345.510582,89286.155270,298.807890,1746,264,1715,1139.054693,103206.433600,1451,-772,240.686940,26893.673580,1187,7640164,83036518,3707533
3,10675,0,151,242,212.870890,323.628662,17.989682,91,990,1039,1008.825276,151.875000,49,118,152.907039,305.053711,70,2900097,65134620,1496356
4,4503,0,-2000,2131,767.000560,561224.468500,749.149163,4131,245,2323,1255.367065,213400.952100,2078,-1227,390.579237,94629.189210,2306,37650551,100860574,9763337


In [4]:
df['Energy_Mean'] = (df['EN_A_F'] + df['EN_A_V'] + df['EN_A_L'])/3
df = df.drop(['EN_A_F','EN_A_V','EN_A_L','Unnamed: 0'], axis = 1)

In [5]:
df.head()

,0,Minimum_A_F,Maximum_A_F,RMS_A_F,Variance_A_F,STD_A_F,RangeA_F,Minimum_A_V,Maximum_A_V,RMS_A_V,Variance_A_V,RangeA_V,Minimum_A_L,RMS_A_L,Variance_A_L,RangeA_L,Energy_Mean
0,0,-606,1000,379.140578,94373.349370,307.202457,1606,431,1529,1105.287236,74150.264650,1098,-465,257.441893,25377.585690,999,3.054259e+07
1,0,131,171,148.916084,77.319336,8.793141,40,960,1000,981.148084,106.616211,40,306,324.583002,94.433594,50,2.325721e+07
2,0,-888,858,345.510582,89286.155270,298.807890,1746,264,1715,1139.054693,103206.433600,1451,-772,240.686940,26893.673580,1187,3.146140e+07
3,0,151,242,212.870890,323.628662,17.989682,91,990,1039,1008.825276,151.875000,49,118,152.907039,305.053711,70,2.317702e+07
4,0,-2000,2131,767.000560,561224.468500,749.149163,4131,245,2323,1255.367065,213400.952100,2078,-1227,390.579237,94629.189210,2306,4.942482e+07


In [6]:
# Split the data for the classifier
y = df['0'].values
X = df.iloc[:,1:]
#X = normalize(X)
X, y = shuffle(X, y, random_state = 2)

In [7]:
# Initiate the 10 - fold split 
kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 2)

# Initiate a decision tree model
clf = DecisionTreeClassifier(class_weight=None, 
                            criterion='entropy',
                            max_depth=10,
                            max_features=3, 
                            max_leaf_nodes=None,
                            min_impurity_decrease=0.0, 
                            min_samples_leaf=18, 
                            min_samples_split=10,
                            min_weight_fraction_leaf=0.0, 
                            #presort=False,
                            random_state=2, 
                            splitter='best'
                            )

# split()  method generate indices to split data into training and test set.
for (train_index, test_index), i in zip(kf.split(X, y), range(10)):
    print(f'Fold:{i}, Train set: {len(train_index)}, Test set:{len(test_index)}')

Fold:0, Train set: 1988, Test set:221
Fold:1, Train set: 1988, Test set:221
Fold:2, Train set: 1988, Test set:221
Fold:3, Train set: 1988, Test set:221
Fold:4, Train set: 1988, Test set:221
Fold:5, Train set: 1988, Test set:221
Fold:6, Train set: 1988, Test set:221
Fold:7, Train set: 1988, Test set:221
Fold:8, Train set: 1988, Test set:221
Fold:9, Train set: 1989, Test set:220


In [17]:
score = cross_val_score(clf, X, y, cv= kf, scoring="accuracy")
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.2f}".format(score.mean())}')

Scores for each fold are: [0.74660633 0.73755656 0.78733032 0.760181   0.76470588 0.76470588
 0.73755656 0.78280543 0.78280543 0.75909091]
Average score: 0.76


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2, stratify = y, test_size = 0.40)

In [9]:
m = clf.fit(X_train, y_train)
y_pred = m.predict(X_test)
accuracy_score(y_test, y_pred)

0.7703619909502263

In [10]:
cnf = confusion_matrix(y_test, y_pred)
print(cnf)

[[ 63  75   4]
 [ 22 615  10]
 [  6  86   3]]


In [11]:
# For class pre-FoG
tp = cnf[1][1]
tn = cnf[0][0] + cnf[0][2] + cnf[2][0] + cnf[2][2]
fp = cnf[1][0] + cnf[1][2]
fn = cnf[0][1] + cnf[2][1]

In [12]:
precision = tp/(tp + fp)
recall =  tp / (tp + fn)    #sensitivity
F1_score = (2*tp) / (2*tp + fp + fn)

In [13]:
print("The evaluation metrics are:\nprecision = {},\nrecall = {}, \nF1-score = {}".format(precision, recall, F1_score) )

The evaluation metrics are:
precision = 0.9505409582689336,
recall = 0.7925257731958762, 
F1-score = 0.8643710470836261


In [15]:
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.69      0.44      0.54       142\n           1       0.79      0.95      0.86       647\n           2       0.18      0.03      0.05        95\n\n    accuracy                           0.77       884\n   macro avg       0.55      0.48      0.49       884\nweighted avg       0.71      0.77      0.73       884\n'